# Projet de programmation

## Introduction <a name="intro"></a>


Ceci est notre projet d'informatique de 2<sup>ème</sup> année de l'ENSAE pour le cours *Python pour un data scientist* où nous avons choisis d'utiliser l'**API de Twitter** et du **sentimental analysis**.

**Problématique** : $\underline{​​\text{​​ En quoi Twitter reflète-t-il la polarisation aux États-Unis, autour des élections présidentielles Américaines ?}​​}​​$

### Installation

Récupération du projet sur GitHub avec :

        git clone https://github.com/gwatkinson/projet-python-twitter.git
        cd projet-python-twitter

Upgrade pip :

        python3 -m pip install --upgrade pip

Créer un environnement virtuel :

        pip install virtualenv
        virtualenv .venv
        source .venv/bin/activate
        # .venv\Scripts\activate # sur Windows

Pour utiliser directement le `setup.py` :

        pip install .

Sinon, installer directement des packages nécessaire :

        pip install -r requirements.txt


Puis, il faut créer le fichier `projet/_credentials.py`, qui contient les clés de l'API de Twitter.

Dans le format suivant :

```python
credentials = {
    "consumer_key": "XXXXXXX",
    "consumer_secret": "XXXXXXX",
    "access_token": "XXXXXXX",
    "access_token_secret": "XXXXXXX",
}
```

Finalement, il faut ajouter les données dans le dossier `data/json/`.

### Modules

* <a href="https://gwatkinson.github.io/projet-python-twitter/projet/streaming.html" target="_blank">streaming</a>
* <a href="https://gwatkinson.github.io/projet-python-twitter/projet/processing.html" target="_blank">processing</a>
* <a href="https://gwatkinson.github.io/projet-python-twitter/projet/modelisation.html" target="_blank">modelisation</a>
* <a href="https://gwatkinson.github.io/projet-python-twitter/projet/visualisation.html" target="_blank">visualisation</a>

### Documentation

<a href="https://gwatkinson.github.io/projet-python-twitter/" target="_blank">https://gwatkinson.github.io/projet-python-twitter/</a>

### Auteurs

* Gabriel Watkinson
* Mathias Vigouroux
* Wilfried Yapi

## Table des matières

* [Introduction](#intro)
* [1)Récupération des données](#data)
* [2)Modélisation](#model)
    * [a.Prepocessing](#process)
    * [b.Clustering](#cluster)
* [3)Visualisation](#visu)
    * [a.Table des États](#states)
    * [b.Carte interactive](#map)
* [Conclusion](#conc)
* [Annexes](#annex)

## 1) Récupération des données <a name="data"></a>



Nous avons utilisé l'**API** de Twitter pour récupérer les nouveaux tweets publiés sur Twitter, la nuit du 3 au 4 Novembre 2020 (la nuit de l'éléction américaine). Nous avons seulement récupérer les tweets qui contennaient certains mots :

```python
# Liste 3 sur Trump et Biden uniquement
liste_3 = [
    "biden",
    "trump",
    "JoeBiden",
    "realDonaldTrump",
]

# Liste 4 sur le thème 'vote'
liste_4 = [
    "iwillvote",
    "govote",
    "uselection",
    "vote",
]

# Liste 5 sur le thème 'election'
liste_5 = [
    "uselection",
    "president",
    "presidentialelection",
    "presidential",
    "electionnight",
]
```

Pour cela, nous avons utilisé le module python `tweepy` ainsi que les fonctions codées dans le module [streaming](https://gwatkinson.github.io/projet-python-twitter/projet/streaming.html) (voir la documentation pour plus d'information sur [start_stream](https://gwatkinson.github.io/projet-python-twitter/projet/streaming.html#projet.streaming.start_stream)). Voici un exemple d'utilisation du code que nous avons écrit :

In [ ]:
import projet.streaming as stream                           # Contient les fonctions pour le streaming
import projet.listes_mots as listes                         # Contient les listes de mots
import projet._credentials as cred                          # Contient les clés d'authentification à l'API

credentials = stream.CredentialsClass(cred.credentials)     # Pour se connecter à l'API (il faut le fichier projet/_credentials.py)

stream.start_stream(
    credentials=credentials,
    liste_mots=listes.liste_3,                              # Liste des mots à tracker (voir `projet.listes_mots`)
    nb=200,                                                 # Nombre de tweets à recupérer
    # timeout=10/3600,                                        # Durée du stream
    fprefix="exemple_liste_3",                              # À modifier en fonction de la liste selectionnée
    path="./data/",                                         # À modifier selon l'utilisateur (doit finir par "/" ou "\")
    verbose=True,
)

Un fichier du format `exemple_liste_3_{date}.json` a été créé dans `data/`.

Pour voir à quoi ressemble les données :

In [ ]:
import glob
import json
import pandas as pd

path = glob.glob("data/exemple_liste_3*.json")[-1]  # On récupère le dernier fichier exemple crée
print("On regarde le fichier : "+path+"\n")

tweets_list = []
with open(path, "r") as fh:
    file = fh.read().split("\n")
    for line in file:
        if line:
            tweets_list.append(json.loads(line))

print("Le premier tweet :")
print(tweets_list[0])


Il s'agit du format `json`. Il est difficile de voir les variable comme cela. On peut créer une `dataframe pandas` pour mieux comprendre les données.

In [ ]:
df_tweets = pd.DataFrame(tweets_list)
df_tweets.head()

In [ ]:
print(f"On voit bien qu'il y a {df_tweets.shape[0]} lignes (une par tweet) et {df_tweets.shape[1]} colonnes (en fait, il y a plus de variables car certaines colonnes sont des dictionnaires).")

In [ ]:
print("Colonnes :\n")
for name in list(df_tweets):
    print(name)

Mais certaine variables sont des dictionnaires (par exemple : `user`, `place`, ...), et il faut donc nettoyer un petit peu la dataframe.

## 2) Modélisation <a name="model"></a>

### a. Preprocessing <a name="process"></a>

Nous avons fait une fonction qui fait les étapes précédentes ainsi que des fonctions pour nettoyer les données. Elles sont dans le fichier [processing](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html).

In [ ]:
import projet.processing as process     # Contient les fonctions pour le processing de la dataframe

Voir [tweet_json_to_df](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.tweet_json_to_df) pour plus de détails.

In [ ]:
import glob
import random

paths = glob.glob("data/json/*.json")
list_path = random.sample(paths, int(len(paths)/3))                      # On récupère un tiers des fichiers json dans le dossier 'data/json/'
# folder = "./data/json/"                                                   # Pour récupèrer tous les fichiers json dans le dossier 'data/json/'

full_df = process.tweet_json_to_df(path_list=list_path, verbose=True)    # Convertit les json en dataframe pandas


Nous avons ainsi récupérer les fichiers dans `data/json/` dans la dataframe pandas `full_df`.

Elle ressemble à :

In [ ]:
full_df.head()

In [ ]:
print(f"On a ainsi récupèrer {full_df.shape[0]} tweets et {full_df.shape[1]} colonnes (il y a en fait plus de variables car certaines colonnes sont des dictionnaires).")

On peut ensuite utiliser `clean_df()` pour filtrer et nettoyer la base de donnée en conservant seulement les informations qui nous interressent. On peut aussi utiliser une liste de `listes_variables` pour récupérer d'autres variables. 

Voir la doc pour plus de détails : [clean_df](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.clean_df).

On créée ainsi la dataframe finale que l'on nomme `df` :

In [ ]:
import projet.listes_variables  # Liste des variables à selectionner

df = process.clean_df(full_df, index="id", date="created_at", verbose=True, columns=projet.listes_variables.liste_1)

In [ ]:
df.head()

Il reste 24 colonnes au lieu de 37. 

On récupère le texte entier, qui se trouve la colonne `extended_tweet-full_text` ou dans `retweeted_status-extended_tweet-full_text`, avec la fonction [get_full_text](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.get_full_text) qui ajoute la colonne `full_text`.

In [ ]:
process.get_full_text(
    df,
    new_var="full_text",
    text_vars=[
        "extended_tweet-full_text",
        "retweeted_status-extended_tweet-full_text",
        "retweeted_status-text",
        "text",
    ],
    drop_vars=True,  # drop_vars=True supprime les anciennes colonnes contenant du texte
)

df["full_text"].head()

Ensuite, on ajoute des colonnes (`full_text-contains_trump`, `full_text-contains_biden`, `user-description-contains_biden` et `user-description-contains_biden`) qui indique la présence de Trump ou de Biden dans le texte ou la description de l'utilisateur. Voir [add_politics](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.add_politics).

In [ ]:
process.add_politics(
    df,
    trump_word="(Trump|Donald|realDonaldTrump|republican)", # Mot pour considerer la présence de Trump
    biden_word="(Biden|Joe|JoeBiden|democrat)", # Pour biden
    case=False, # case sensitive
    trump_var="contains_trump",
    biden_var="contains_biden",
    text_vars=["full_text", "user-description"],
)

df[["full_text", "full_text-contains_trump", "full_text-contains_biden"]].head()

En utilisant `SentimentIntensityAnalyzer` du module `sentiment.vader` de la librairie `nltk`, on ajoute les colonnes qui contiennent le sentiment compound (compris entre -1 et 1) du `full_text` et de `user-description`. Voir [add_sentiment](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.add_sentiment).

In [ ]:
process.add_sentiment(
    df,
    text_vars=["full_text", "user-description"],
    sent_var="sentiment",
    compound_var="compound",
    keep_dict=False,
)

df[["full_text", "full_text-sentiment-compound"]].head()

In [ ]:
df["lang"].head()

In [ ]:
print("Pourcentage de tweet non anglais : ", round(100*len(df[df["lang"]!="en"])/len(df)), "%")

On voit qu'il y a un pourcentage important de tweets étrangers.

On conserve seulement les tweets en anglais. Voir [keep_lang](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.keep_lang).

In [ ]:
df_en = process.keep_lang(df, lang_var="lang", language="en")

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15,5), sharey=True)

axs[0].hist(df_en["full_text-sentiment-compound"], color="g", bins=20)
axs[1].hist(df_en[df_en["full_text-contains_trump"] & ~df_en["full_text-contains_biden"]]["full_text-sentiment-compound"], color="r", bins=20)
axs[2].hist(df_en[~df_en["full_text-contains_trump"] & df_en["full_text-contains_biden"]]["full_text-sentiment-compound"], color="b", bins=20)
fig.suptitle("Histogramme du sentiment compound de full_text")
axs[0].set_title("Tous les tweets", color="g")
axs[1].set_title("Contenant Trump", color="r")
axs[2].set_title("Contenant Biden", color="b")
plt.show()
fig.savefig("image/maps/stats_desc_old.jpg")

On remarque qu'il y a beaucoup de compound nuls. Cela peut être du au fait qu'il y a un nombre important de tweets très courts (émojis, numerique).

On les enlève pour mieux voir la polarisation (voir [remove_null](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.remove_null)) :

In [ ]:
df_null = process.remove_null(df_en)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15,5), sharey=True)

axs[0].hist(df_null["full_text-sentiment-compound"], color="g", bins=20)
axs[1].hist(df_null[df_null["full_text-contains_trump"] & ~df_null["full_text-contains_biden"]]["full_text-sentiment-compound"], color="r", bins=20)
axs[2].hist(df_null[~df_null["full_text-contains_trump"] & df_null["full_text-contains_biden"]]["full_text-sentiment-compound"], color="b", bins=20)
fig.suptitle("Histogramme du sentiment compound de full_text")
axs[0].set_title("Tous les tweets", color="g")
axs[1].set_title("Contenant Trump", color="r")
axs[2].set_title("Contenant Biden", color="b")
plt.show()
fig.savefig("image/maps/stats_desc.jpg")

On peut ensuite rajouter un label aux compounds pour les discrétiser. Voir [sentiment_class](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.sentiment_class).

In [ ]:
process.sentiment_class(
    df_null,
    categories=[
        ("tneg", -1, -0.7),
        ("neg", -0.7, -0.2),
        ("neutre", -0.2, 0.2),
        ("pos", 0.2, 0.7),
        ("tpos", 0.7, 1),
    ],
    compound_vars=[
        "full_text-sentiment-compound",
        "user-description-sentiment-compound",
    ],
    class_var="class",
)

df_null[["full_text-sentiment-compound", "full_text-sentiment-compound-class"]].head()

On ajoute ensuite un label qui joint le sentiment du tweet et la présence de Biden ou Trump. Voir [add_label](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.add_label).

In [ ]:
process.add_label(
    df_null,
    label_var="label",
    trump_var=("full_text-contains_trump", "T"),
    biden_var=("full_text-contains_biden", "B"),
    missing_var="N",
    class_var="full_text-sentiment-compound-class",
)

df_null[["full_text-contains_trump", "full_text-contains_biden", "full_text-sentiment-compound-class", "label"]].head()

Finalement, on peut ajouter les États à partir de la description de la localisation fournie par les utilisateurs.

On a essayé trois manières de les obtenir :

* Utiliser les coordonnées données par la variable `place` du tweet pour determiner l'état. Cependant, moins de 1% des tweets possèdent cette variable.

* Utiliser une librairie de NLP pour reconnaître la présence d'État ou d'une ville dans la chaîne de caractère données par l'utilisateur (`user-location`). Cependant, la librairie que l'on a trouvé est lente et pas significativement plus efficace que la troisième méthode qui est plus simple. Voir la fonction [get_states1](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.get_states1) pour plus de détails.

* Finalement, on a décider d'utiliser des expressions régulières pour identifier le nom des États. Voir [get_states](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.get_states).
    

In [ ]:
pc = df_null["place-bounding_box-coordinates"].count() / len(df_null)
print(f"Il y a{pc*100: .1f}% de tweets qui donnent leur position avec la première méthode.")

In [ ]:
process.get_states(df_null, state_var="state", location_var="user-location")

df_null[["user-location", "state"]].head(10)

In [ ]:
pc2 = df_null["state"].count() / len(df_null)
print(f"Il y a{pc2*100: .1f}% de tweets qui donnent leur position avec la troisième méthode.")

On conserve uniquement les tweets avec une position (voir [keep_states](https://gwatkinson.github.io/projet-python-twitter/projet/processing.html#projet.processing.keep_states)):

In [ ]:
df_final = process.keep_states(df_null, state_var="state")

df_final[["user-location", "state"]].head()

In [ ]:
n_init = len(full_df)
n_final = len(df_final)

print(f"Il reste {n_final} des {n_init} tweets initiaux après le nettoyage de la dataframe.")
print(f"Soit {100*n_final/n_init :.1f}%.")

### b. Clustering <a name="cluster"></a>

On passe à la partie sur le clustering.

In [ ]:
import projet.modelisation as model     # Contient les fonctions pour le clustering

On fait du K-means. On standardise une copie de la df puis ajoute une colonne label à df principale.

On le fait deux fois :
* La première utilise la méthode du coude pour déterminer le nombre de clusters et utilise toutes les variables numériques (sauf l'id).

* La seconde a un nombre de clusters détérminés (6), et utilise seulement quelques variables.

Voir la doc pour plus de détails : [KM](https://gwatkinson.github.io/projet-python-twitter/projet/modelisation.html#projet.modelisation.KM).

In [ ]:
model.KM(
    df_final,
    n_cluster=None,             # Prend la valeur optimal par la méthode du coude
    label_var="kmlabel_opt",    # Nom de la nouvelle colonne
    vars=None,                  # Prend les valeurs numériques et booléennes sans l'id
    drop_vars=["user-id"],
    n_init=10,                  # Nombre de fois que le k-means est lancé
    max_iter=300,               # Nombre max d'itérations
    max_cluster=10,             # Nombre max de clusters
    random_state=20,
    plot=True,                  # Plot le SSE
)

print()

<a name="kmclusters"></a>

In [ ]:
df_final.groupby(["kmlabel_opt", "label"]).describe()["user-id"]["count"]

On voit au dessus le nombre de tweets par label selon le cluster détérminé par le 1<sup>er</sup> kmeans.

Cette répartition ne correspond pas au sentiment du tweet selon la présence de Trump ou Biden, notamment parcequ'on prend en compte la popularité des utilisateurs (nombre de followers, d'amis, ...).

Donc on considère un autre modèle avec moins de variables :

In [ ]:
model.KM(
    df_final,
    n_cluster=6,                                # 6 clusters
    label_var="kmlabel",                        # Nom de la nouvelle colonne
    vars=[
        'full_text-contains_trump',             # Selection des variables pour le clusters
        'full_text-contains_biden',
        'full_text-sentiment-compound',
        'user-description-sentiment-compound'
    ],
    n_init=10,                                  # Nombre de fois que le k-means est lancé
    max_iter=300,                               # Nombre max d'itérations
    random_state=30,
)

print()

In [ ]:
df_final["kmlabel"].value_counts().plot(kind="hist", title="Histogramme des labels")

In [ ]:
df_final.groupby(["kmlabel", "label"]).describe()["user-id"]["count"]

On peut identifier les clusters à des groupes de tweets homogènes d'un point de vue du sentiment politique.

Par exemple, un cluster des tweets qui supportent Trump, ceux qui supportent Biden, ceux qui haïssent Trump, ect...

## 3) Visualisation <a name="visu"></a>

In [ ]:
import projet.visualisation as visu     # Contient les fonctions pour la visualisation

### a. Table des États <a name="states"></a>

On créée une dataframe geopandas qui contient la forme des États américains (voir [create_gdf](https://gwatkinson.github.io/projet-python-twitter/projet/visualisation.html#projet.visualisation.keep_states)) :

In [ ]:
gdf = visu.create_gdf()

In [ ]:
gdf.head()

### b. Carte interactive <a name="map"></a>

On créée les images des histogrammes par État dans `images/`. Voir [save_hist](https://gwatkinson.github.io/projet-python-twitter/projet/visualisation.html#projet.visualisation.keep_states).

On ajoute le cluster majoritaire dans chaque État. Voir [add_max](https://gwatkinson.github.io/projet-python-twitter/projet/visualisation.html#projet.visualisation.add_max).

Finalement, on ajoute des stats (total, moyenne, écart-type) du sentiment compound par État.

In [ ]:
visu.save_hist(df_final, gdf, label="kmlabel")  # Ajoute les histogrammes dans le dossier image/hist/kmlabel
visu.save_hist(df_final, gdf, label="label")
visu.add_max(df_final, gdf, label="label")      # Ajoute le cluster majoritaire de chaque état
visu.add_max(df_final, gdf, label="kmlabel")
gdf2 = visu.add_stats_sentiment(df_final, gdf)         # Ajoute des stats sur le sentiment compound du full_text

gdf2.head()

On affiche les stats par État :

In [ ]:
fig2, axs2 = plt.subplots(1, 3, figsize=(24,7))
fig2.suptitle("Stats du sentiment compound par État")
mask = (gdf2["NAME10"]!="Alaska") & (gdf2["NAME10"]!="Hawaii")
gdf2[mask].plot(column="count", legend=True, ax=axs2[0], legend_kwds={'label': "Nombre de tweets par État", 'orientation': "horizontal"})
gdf2[mask].plot(column="mean", legend=True, ax=axs2[1], legend_kwds={'label': "Moyenne du sentiment compound par État", 'orientation': "horizontal"})
gdf2[mask].plot(column="std", legend=True, ax=axs2[2], legend_kwds={'label': "Écart-type du sentiment compound par État", 'orientation': "horizontal"})
plt.show()

In [ ]:
fig2.savefig("image/maps/stats.jpg")

Les couleurs désignent le cluster majoritaire dans l'État. Voir [les clusters](#kmclusters).

On affiche les cartes interactives (voir [plot_hist](https://gwatkinson.github.io/projet-python-twitter/projet/visualisation.html#projet.visualisation.plot_hist)):

In [ ]:
visu.plot_hist(gdf2, label="kmlabel")

In [ ]:
visu.plot_hist(gdf2, label="label")

## Conclusion <a name="conc"></a>

Grâce au cartes, on visualise bien la polarisation des États-Unis, notamment autour du sujet des éléctions américaines et des candidats Donald Trump et Joe Biden. 